In [38]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

In [44]:
df_general = pd.read_csv("/home/bosser/Documentos/PROYECTOFINAL/CSV/CSV_finales/ofertas_final.csv")
df_general_ciudades = df_general[["id", "ciudad"]]  # df con las columnas necesarias.

def split_presencial(row):
    if pd.notna(row["ciudad"]) and (" OR " in row["ciudad"] or " O " in row["ciudad"]):
        ciudades = [ciudad.strip() for ciudad in row["ciudad"].replace(" O ", " OR ").split(" OR ")]
        return pd.DataFrame({
            "id": [row["id"]] * len(ciudades),
            "ciudades": ciudades})
    else:
        return pd.DataFrame({"id": [row["id"]], "ciudades": [row["ciudad"]]})

#Aplicamos la función, concatenamos el resultado y generamos csv
df_ciudades = pd.concat(df_general_ciudades.apply(split_presencial, axis=1).tolist(), ignore_index=True)
df_ciudades.to_csv("resultado_presencial.csv", index=False) ###### PATH y nombre del archivo.

In [ ]:
ciudades = list(set(df_ciudades["ciudades"]))
def obtener_coordenadas(ciudades):
    
    # Inicializamos el geolocalizador
    geolocalizador = Nominatim(user_agent="geoapi")
    # Lista para almacenar los datos
    datos = []

    for ciudad in ciudades:
        try:
            ubicacion = geolocalizador.geocode(ciudad, timeout=10, addressdetails=True)
            if ubicacion:
                direccion = ubicacion.raw.get("address", {})
                datos.append({
                    "Ciudad": ciudad.title(),
                    "País": direccion.get("country", "Desconocido"),
                    "Comunidad Autónoma": direccion.get("state", "Desconocido"),
                    "Latitud": ubicacion.latitude,
                    "Longitud": ubicacion.longitude})
            else:
                print(f"No se encontraron coordenadas para: {ciudad}")
            sleep(1)  # Esperar para evitar bloqueos de la API
        except Exception as e:
            print(f"Error con {ciudad}: {e}")

    # Convertimos a DataFrame e ignoramos filas con nan
    df = pd.DataFrame(datos).dropna()

    # Guardamos en CSV
    df.to_csv("ciudades_coordenadas.csv", index=False)   ###### PATH  

obtener_coordenadas(ciudades)